<a href="https://colab.research.google.com/github/WilliamPoe/CSCI-290/blob/main/notebooks/asm_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import pandas as pd
import numpy as np
import math
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

In [24]:
def classification(df, target, method):
  if method == "gini":
    gini = 0
    lowest_gini = float('inf')
    split_point = 0
    lowest_quant_gini = float('inf')
    quant_attrib = ''
    overall = len( df )
    for attrib in df.columns: ### change this to df.columns ###
      vals = df[attrib].unique()
      # Quantitative Gini Impurity Calc
      if df[attrib].dtype in ['float64', 'int64']:
        if len( df[attrib].unique() ) > 10:
          vals.sort()
          overall = len( df )
          gini_impurities = []
          for val in vals:
            quant_gini = 0
            left = df[ df[attrib] <= val ][ [attrib,target] ]
            props = left[ target ].value_counts( normalize = True )
            weight = len( left ) / overall
            for prop in props.array:
              quant_gini += weight*(1-(prop**2))
            right = df[ df[attrib] > val ][ [attrib,target] ]
            props = right[ target ].value_counts( normalize = True )
            weight = len( right ) / overall
            for prop in props.array:
              if prop > 0:
                ### Change calculation ###
                quant_gini += weight*(1-(prop**2))
              gini_impurities.append(quant_gini)
          index = pd.Series( gini_impurities ).idxmin()
          # Check to see if there is a new lowest Gini Impurity
          if gini_impurities[index] < lowest_quant_gini:
            # Lowest Gini Impurity
            lowest_quant_gini = gini_impurities[index]
            # Split point with lowest Gini Impurity
            split_point = vals[index]
            # Attribute that has the split point
            quant_attrib = attrib

      # Categorical Gini Impurity Calc
      else:
        for val in vals:
          subset_size = len(df[ df[attrib] == val ])
          weight = subset_size / overall
          props = df[ df[attrib] == val ][target].value_counts( normalize=True )
          for p in props.array:
            gini +=  weight*(1-(p**2))
        if lowest_gini > gini:
          lowest_gini = gini
          lowest_attrib = attrib
    # Compares the lowest categorical Gini Impurity with the lowest quantitative Gini Impurity
    if lowest_gini < lowest_quant_gini:
      # If categorical Gini Impurity is still lowest
      return lowest_attrib, lowest_gini
    else:
      # If quantitative Gini Impurity is lowest
      lowest_gini = lowest_quant_gini
      lowest_attrib = quant_attrib
      return lowest_attrib, split_point, lowest_gini
  elif method == "entropy":
    entropy = 0
    lowest_entropy = float('inf')
    split_point = 0
    lowest_quant_entropy = float('inf')
    quant_attrib = ''
    overall = len( df )
    for attrib in df.columns: ### change this to df.columns ###
      vals = df[attrib].unique()
      # Quantitative Entropy Calc
      if df[attrib].dtype in ['float64', 'int64']:
        if len( df[attrib].unique() ) > 10:
          vals.sort()
          overall = len( df )
          entropies = []
          for val in vals:
            quant_entropy = 0
            left = df[ df[attrib] <= val ][ [attrib,target] ]
            props = left[ target ].value_counts( normalize = True )
            weight = len( left ) / overall
            for prop in props.array:
              quant_entropy = quant_entropy - weight*prop*math.log2( prop )
            right = df[ df[attrib] > val ][ [attrib,target] ]
            props = right[ target ].value_counts( normalize = True )
            weight = len( right ) / overall
            for prop in props.array:
              if prop > 0:
                quant_entropy = quant_entropy - weight*prop*math.log2( prop )
              entropies.append(quant_entropy)
          index = pd.Series( entropies ).idxmin()
          # Check to see if there is a new lowest Entropy
          if entropies[index] < lowest_quant_entropy:
            # Lowest entropy
            lowest_quant_entropy = entropies[index]
            # Split point with lowest entropy
            split_point = vals[index]
            # Attribute that has the split point
            quant_attrib = attrib

      # Categorical Entropy Calc
      else:
        for val in vals:
          subset_size = len(df[ df[attrib] == val ])
          weight = subset_size / overall
          props = df[ df[attrib] == val ][target].value_counts( normalize=True )
          for p in props.array:
            entropy =  entropy - weight*(p*math.log2(p))
        if lowest_entropy > entropy:
          lowest_entropy = entropy
          lowest_attrib = attrib
    # Compares the lowest categorical Entropy with the lowest quantitative Entropy
    if lowest_entropy < lowest_quant_entropy:
      # If categorical entropy is still lowest
      return lowest_attrib, lowest_entropy
    else:
      # If quantitative entropy is lowest
      lowest_entropy = lowest_quant_entropy
      lowest_attrib = quant_attrib
      return lowest_attrib, split_point, lowest_entropy
  else:
    print("Invalid Method")
    return

In [25]:
def regression(df, target):
  return

In [28]:
def Attribute_selection_method(task, df, target):
  ##train_set, test_set = train_test_split(df, test_size=0.2)

  ### Add a Pipeline to process the data ###

  if task.lower() == "classification":
    method = input("Enter the method of attribute selection (gini or entropy): ").lower()
    print(classification(df, target, method))
  elif task.lower() == "regression":
    print(regression(df, target))
  else:
    print("Invalid Task")
    return

In [29]:
df = pd.read_csv("https://raw.githubusercontent.com/WilliamPoe/CSCI-290/refs/heads/main/Data/ds_salaries.csv") ## Dataset
target = "employee_residence" ## Target attribute
task = "classification" ## Classification or Regression
Attribute_selection_method(task, df, target)

Enter the method of attribute selection (gini or entropy): entropy
('salary_in_usd', 111000, 0.2559790811041855)
